http://jira.lge.com/issue/browse/HEDATAPLFM-1119

In [0]:
df = spark.read.csv(
    "/Volumes/sandbox/z_najin_lee/v_najin_lee/BootingTest_US_webOS24.csv",
    header=True,
    inferSchema=True,
)

df.show(10)

In [0]:
from pyspark.sql.functions import (
    sha2,
    concat_ws,
    current_timestamp,
    when,
    col,
    lit,
    sha2,
    concat,
    upper,
)

he_etl_dt = current_timestamp()
tv_salt = dbutils.secrets.get("admin", "salt")

# Mac 칼럼 대문자로 변환
df = df.withColumn("mac", upper(col("mac")))

# df_private 컬럼 추가
df = df.withColumn(
    "mac_addr",
    when(col("mac").isNull() | (col("mac") == ""), col("mac")).otherwise(
        sha2(concat(col("mac"), lit(tv_salt)), 256)
    ),
)

df = df.withColumn("he_etl_dt", he_etl_dt)

display(df)

In [0]:
df = df.drop("mac")
display(df)

In [0]:
# df.write.format("delta").mode("overwrite").saveAsTable("sandbox.z_najin_lee.BootingTest_US_webOS24_heds_1119")

In [0]:
# spark.sql("DROP TABLE IF EXISTS sandbox.z_najin_lee.BootingTest_US_webOS24_heds_1119")

In [0]:

df.write.format("delta").mode("overwrite").saveAsTable("sandbox.z_najin_lee.BootingTest_US_webOS24_heds_1097_upper")

In [0]:
%sql
select *
from sandbox.z_najin_lee.BootingTest_US_webOS24_heds_1119
limit 20;

In [0]:
%sql
select distinct mac_addr
from sandbox.z_najin_lee.BootingTest_US_webOS24_heds_1119
limit 20;

In [0]:
mac_list = [row.Mac for row in df.select("Mac").distinct().collect()]

mac_list_upper = [row.Mac.upper() for row in df.select("Mac").distinct().collect()]


In [0]:
from pyspark.sql.functions import col

df = spark.table("aic_data_private.tlamp.rbt_mac_addr_webos24")
filtered_df = df.filter(col("mac_addr").isin(mac_list_upper))
display(filtered_df)

In [0]:
mac_list_upper

In [0]:
%sql
select *
from aic_data_private.tlamp.rbt_mac_addr_webos24
where 1=1
  -- and mac_addr like '74:c1%'
  and mac_addr like '00:A1%'